In [6]:
#lilo
%cd '/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks'

/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks


In [5]:
!git config --global user.email yossi.cohen@live.com”
!git config --global user.name “Yossi Cohen”
# !git commit -am "poisson-exp-3" && git push


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@87271ef43271.(none)')
